In [ ]:
from keras.callbacks import TensorBoard

In [ ]:
import pandas as pd
import numpy as np
import scipy
import sys
import imp

In [ ]:
from process_data import load_data,get_rank_of_size
from callbacks import MetricsEx
from data_util import unpack_news_data,data_generator
import models
imp.reload(models)

In [ ]:
rank = get_rank_of_size()
rank.remove('GOOG')

In [ ]:
print(rank)

In [ ]:
data = load_data(rank[:10],encoder_kind='Bert')

In [ ]:
# sparse matrix
data[rank[0]]['train'][0][0][0].toarray().shape

# TEXTUAL MODEL

In [ ]:
model = models.build_textual_model()

In [ ]:
print(model.summary())

In [ ]:
b_size = 32
x_train = np.concatenate([data[key]['train'][0] for key in data],axis=0)
y_train = np.concatenate([data[key]['train'][2] for key in data],axis=0)
x_val = unpack_news_data(np.concatenate([data[key]['val'][0] for key in data],axis=0))
y_val = np.concatenate([data[key]['val'][2] for key in data],axis=0)
cbs = [TensorBoard(log_dir='model_log/'),MetricsEx('f1')]
steps = (len(x_train)+b_size-1)//b_size

In [ ]:
model.fit_generator(generator=data_generator(b_size,(x_train,y_train),unpack_news_data),
                    samples_per_epoch=steps,
                    validation_data=(x_val,y_val),
                    callbacks = cbs,
                    epochs=4,
                    verbose=1,
                   )

In [ ]:
model.evaluate(x=unpack_news_data(data[rank[2]]['val'][0]), y=data[rank[2]]['val'][2],batch_size=32)

# NUMERICAL MODEL

In [ ]:
nmodel = models.build_numerical_model()

In [ ]:
print(nmodel.summary())

In [ ]:
b_size = 32
x_train = np.concatenate([data[key]['train'][1] for key in data],axis=0)
y_train = np.concatenate([data[key]['train'][2] for key in data],axis=0)
x_val = np.concatenate([data[key]['val'][1] for key in data],axis=0)
y_val = np.concatenate([data[key]['val'][2] for key in data],axis=0)
cbs = [TensorBoard(log_dir='num_log/'),MetricsEx('f1')]
steps = (len(x_train)+b_size-1)//b_size

In [ ]:
nmodel.fit_generator(generator=data_generator(b_size,(x_train,y_train)),
                    samples_per_epoch=steps,
                    validation_data=(x_val,y_val),
                    callbacks = cbs,
                    epochs=50,
                    verbose=1,
                   )

# Hybrid model

In [ ]:
hmodel = models.build_hybrid_model()

In [ ]:
b_size = 32
x1_train = np.concatenate([data[key]['train'][0] for key in data],axis=0)
x2_train = np.concatenate([data[key]['train'][1] for key in data],axis=0)
x1_val = unpack_news_data(np.concatenate([data[key]['val'][0] for key in data],axis=0))
x2_val = np.concatenate([data[key]['val'][1] for key in data],axis=0)
y_val = np.concatenate([data[key]['val'][2] for key in data],axis=0)
y_train = np.concatenate([data[key]['train'][2] for key in data],axis=0)
cbs = [TensorBoard(log_dir='num_log/')]
steps = (len(x1_train)+b_size-1)//b_size

In [ ]:
hmodel.fit_generator(generator=data_generator(b_size,(x1_train,x2_train,y_train),unpack_news_data),
                    samples_per_epoch=steps,
                    validation_data=([x1_val,x2_val],y_val),
                    callbacks = cbs,
                    epochs=10,
                    verbose=1,
                   )

In [ ]:
result_dict = {}
for code in rank[:20]:
    result = hmodel.evaluate(x=[unpack_news_data(data[code]['test'][0]),data[code]['test'][1]],
                             y=data[code]['test'][2],batch_size=32)
    result_dict[code] = result[1]
    if 'avg' not in result_dict:
        result_dict['avg'] = 0
    result_dict['avg'] += result[1]
result_dict['avg'] /= 20
print(result_dict)

# Random Guess